In [1]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression as LR
from sklearn.model_selection import train_test_split

from sklearn.metrics import r2_score as r2, mean_squared_error as mse

In [2]:
# источник - https://russia.duck.consulting
df_unemp = pd.read_csv('input/unemp.csv', sep=';', encoding='cp1251')
df_salary = pd.read_csv('input/salary.csv', sep=';', encoding='cp1251')
df_crime = pd.read_csv('input/crime_per_10000.csv', sep=';', encoding='cp1251')

## Задача 1: Baseline model

In [3]:
df = df_unemp.merge( df_salary, how='inner', on=['year', 'region'])  #.to_csv('res.csv', sep=';', encoding='cp1251')

In [4]:
df = df.merge( df_crime, how='inner', on=['year', 'region']) 

In [5]:
df['log_salary'] = np.log(df['salary'])

In [6]:
df.head()

,year,region,unemp_rate,salary,crime_per,log_salary
0,2009,Адыгея,7.7,10798.0,110.72,9.287116
1,2009,Алтайский край,12.3,10741.0,222.12,9.281823
2,2009,Амурская область,8.3,17969.0,237.12,9.796403
3,2009,Архангельская область,7.2,17654.0,213.81,9.778718
4,2009,Астраханская область,9.9,14011.0,298.37,9.547598


In [7]:
train, valid = train_test_split(df, test_size=0.3, random_state=70)

In [8]:
train.shape, valid.shape

((292, 6), (126, 6))

In [9]:
def u(x):
    """ utility function """
    return np.sqrt(np.sqrt(x))

In [10]:
model = LR()

In [11]:
feats = ['unemp_rate', 'salary']
y = 'crime_per'

In [12]:
model.fit(df[feats], df[y])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [13]:
pred_trn = model.predict(train.loc[:, feats])
r2_trn = r2(train[y], pred_trn)
    
pred_tst = model.predict(valid.loc[:, feats])
r2_tst = r2(valid[y], pred_tst)

In [14]:
r2_trn, r2_tst

(0.002696438932272116, 0.014909296114972048)

### Вывод: Эксперименты на различных выпуклых функциях полезности и наборах признаков показывают низкий уровень объясняющей способности выбранных независимых переменных для уровня преступности. В силу линейности формулы Бейкера проверка нелинейных моделей не требуется. Однако делать вывод о работоспособности формулы Бейкера нужно после анализа других параметров, заложенных в ней.

## Задача 2: User model

### Для исследования будут использованы следующие допущения:

1) Формула Беккера оказывает влияние именно на экономическую группу преступлений, связанную с оценкой потенциальным преступником возможности обогащения в результате преступления

2) Будем использовать официальную статистику о раскрываемости преступлений как основу для вычисления вероятности наказания

3) Преобразуем неравенство Беккера в неравенство с нулевой правой частью и будем считать вероятность выполнения неравенства как сигмоидную функцию от правой части

4) Полученную вероятность будем использовать в качестве единственной зависимой переменной для линейной регрессионнной модели


#### В качестве целевой переменной будем использовать процент экономических преступлений

In [15]:

df_ec_crime = pd.read_csv('input/ec_crime.csv', sep=';', encoding='cp1251')

In [16]:
df = df.merge( df_ec_crime, how='inner', on=['year', 'region']) 

#### Добавим процент нераскрытых преступлений для вычисления вероятности наказания

In [17]:

df_unsolved = pd.read_csv('input/unsolved.csv', sep=';', encoding='cp1251')

In [18]:
df = df.merge( df_unsolved, how='inner', on=['year', 'region']) 

In [19]:
df.shape

(418, 8)

#### Эффективное "психологическое" представление о вероятности наказания - ниже официальной отчетности МВД.  Введем условный коэффициент 1/5

In [20]:
df['eff_solved'] = (100 - df['unsolved'])/500

In [21]:
def p_bekker(gain, fine, xdf):
    # преобразование формулы Беккера в вероятность (психологическую готовность) при помощи сигмоидной функции
    e = (1 + 
          np.exp(
                 - (1 - xdf['eff_solved'])*u(gain) +  
                 xdf['eff_solved']*u(fine) +
                 u(xdf['salary'])*(100 - xdf['unemp_rate'])/100  # эффективная полезность легального дохода с учетом безработицы
                )
         )
    xdf['p_bekker'] = 1 / e

#### Вычисляем психологическую готовность для каждой записи датасета 
#### - преступный выигрыш - 100 000 р., 
#### - штраф - 2 500 000 р.

In [22]:
p_bekker(100000, 2500000, df)

In [23]:
train, valid = train_test_split(df, test_size=0.3, random_state=70)

In [24]:
train.shape, valid.shape

((292, 10), (126, 10))

#### Обучаем модель только на беккеровой вероятности

In [25]:
feats = [ 'p_bekker']
y = 'ec_crime_per'

In [26]:
model.fit(df[feats], df[y])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [27]:
pred_trn = model.predict(train.loc[:, feats])
r2_trn = r2(train[y], pred_trn)
    
pred_tst = model.predict(valid.loc[:, feats])
r2_tst = r2(valid[y], pred_tst)

In [28]:
r2_trn, r2_tst

(0.21907580580308306, 0.14278933144011463)

### Вывод: принятые при построении пользовательской модели допущения позволили на несколько порядков повысить долю объясненной дисперсии. Недостаточно высокий ее абсолютный уровень можно объяснить константным характером оценки ожидаемой выгоды и штрафа, которые могут быть различны в разных регионах и зависят от экономического состояния этих регионов. Можно предложить уточнение модели в этом направлении.